In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt


In [2]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## Loading detection results and preparing label map (from class ID to class title)

In [3]:
with open('../../results/det_results_merged_34b.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

In [4]:
with open('../../../wsod/metadata/ont_m18/class_names_all.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [5]:
all_classes = list(mid2name_all)
mid2idx = {key: i for i, key in enumerate(all_classes)}
category_index = {i:{'id': i, 'name': mid2name_all[key]} for i, key in enumerate(all_classes)}

## Visualizing Results

In [6]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [7]:
with open('../../../../data/eval_m18/kf_id2path.pkl', 'rb') as fin:
    kf_id_to_img_path = pickle.load(fin)

In [15]:
def show(imgid, thresh):
    filename = kf_id_to_img_path[imgid]
    print(imgid)
    with open(filename, 'rb') as fin:
        if filename.endswith('.ldcc'):
            _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.frombuffer(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = det_results_merged[imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx[det['label']] for det in detections]
        
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=thresh-0.001,
      max_boxes_to_draw=100,
      line_thickness=2)

    cv2.imwrite('temp.jpg', image_np)

In [9]:
images = list(kf_id_to_img_path.keys())

In [16]:
imgid = np.random.choice(images)
show(imgid, 0.0)
    

HC000TKZY_27
